In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dts
import scipy.stats as stats
import yfinance as yfin

# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score

# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

import requests
import xlsxwriter
from datetime import date

In [ ]:
# https://towardsdatascience.com/historical-stock-price-data-in-python-a0b6dc826836
# https://towardsdatascience.com/free-stock-data-for-python-using-yahoo-finance-api-9dafd96cad2e

In [3]:
## 1. 3 candle stick(CS)[-ve] for the past 3 days( 4 days inlcuding the current day)
## 2. no. of 3CS pattern for last 3/6 months and immediate increase in %age
## 3. 2CS[+ve] for past 2 day(3 days including the current day)
## 4. calcualte 20 DMA and 50DMA and compare it with CMP
## 5. get the PE data and sector 
## 6. Calculate beta for each stock
## 7. use webscrapping to get latest news related to the stock

# 3 Candle Stick Pattern

In [4]:
def percDiff(a,b):
    perc= ((b-a)*100)/a
    return perc

# NIFTY STOCKS

In [5]:
def stockName(fileName):
    data=pd.read_csv(fileName)
    stocks=data['Symbol']
    for i in range(0,len(stocks)):
        stocks[i]=stocks[i]+'.NS'
    return stocks

In [32]:
def nifty_delta(fileName):
    stocks=stockName(fileName)
    # stocks=['INFY.NS','IBREALEST.NS']
    TPE=[] #trailing PE
    SECTOR=[]
    CMP=[]
    fiftyMovAvg=[]
    three_CS=[]
    delta=[]
    perc=0
    dict_delta={}
    for i in range(0,1):
        perc=0
        ticker=yfin.Ticker(stocks[i])
        lst=ticker.info.keys()
        trend=[]
        data=ticker.history(start='2021-03-15',end='2021-03-16')
        CMP.append(ticker.info['regularMarketPrice'])
        SECTOR.append(ticker.info['sector'])
        if (list(lst).count('fiftyDayAverage')!=0):
            fiftyMovAvg.append(ticker.info['fiftyDayAverage'])
        else:
            fiftyMovAvg.append('NaN')
        if (list(lst).count('trailingPE')!=0):
            TPE.append(ticker.info['trailingPE'])
        else:
            TPE.append('NaN')

        for l in range(0,len(data)):
            perc+=percDiff(data['Open'][l],data['Close'][l])
            trend.append(perc)
        delta.append(perc)
        
        iCnt=0 ## to count the continuous DOwn trends, 1->3 red candle sticks
        print(len(data))
        print(data)
        for k in range(0,len(data)):
            if(k+3<len(data)):
                print('hi')
                if(trend[k+1]<0 and trend[k+2]<0 and trend[k+3]<0):
                    iCnt+=1
        print(iCnt)
        if iCnt==1:
            three_CS.append('Yes')
        else:
            three_CS.append('No')
            
    dict_delta={'Stocks':stocks,'Trailing PE':TPE,'Delta':delta,'CMP':CMP,'50DMA':fiftyMovAvg,'Sector':SECTOR,'3 Red CS':three_CS}
    return dict_delta

In [33]:
dict_delta=nifty_delta('NIFTY50.csv')
# df_nifty50=pd.DataFrame(dict_delta)
# df_nifty50

C:\Users\PRADEEP\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


- SBIN.NS: No data found for this date range, symbol may be delisted
0
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
0


In [19]:
df_nifty50

,Stocks,Trailing PE,Delta,CMP,50DMA,Sector,3 Red CS
0,SBIN.NS,14.5641,-4.534478,385.60,371.805880,Financial Services,No
1,TATASTEEL.NS,NaN,-4.878082,740.10,692.736760,Basic Materials,No
2,DIVISLAB.NS,47.9463,-2.084781,3429.90,3568.808800,Healthcare,No
3,KOTAKBANK.NS,40.1443,-3.353369,1925.10,1895.891200,Financial Services,No
4,DRREDDY.NS,34.8109,-0.479577,4380.10,4636.186500,Healthcare,No
5,ULTRACEMCO.NS,27.4132,-1.160619,6590.00,6241.551300,Basic Materials,No
6,ITC.NS,19.2739,-0.643630,206.40,214.239700,Consumer Defensive,No
7,CIPLA.NS,28.5936,-2.881962,805.10,823.072100,Healthcare,No
8,HDFCBANK.NS,27.1632,-4.227898,1530.90,1546.248500,Financial Services,No
9,HINDUNILVR.NS,70.5765,-0.375686,2221.35,2224.044200,Consumer Defensive,No


In [31]:
dict_delta_Midcap=nifty_delta('MIDCAP50_Yahoo.csv')
df_nifty_midcap=pd.DataFrame(dict_delta_Midcap)
df_nifty_midcap

C:\Users\PRADEEP\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Stocks,Trailing PE,Delta
0,MGL.NS,21.0777,-1.669935
1,BATAINDIA.NS,NaN,-3.720926
2,TVSMOTOR.NS,78.5819,-7.804939
3,MFSL.NS,74.9874,-6.010990
4,CASTROLIND.NS,21.528,-8.614853
5,ADANIENT.NS,131.427,-5.153487
6,GLENMARK.NS,13.9686,-1.168301
7,RECLTD.NS,4.45164,3.902297
8,MINDTREE.NS,31.8448,9.863015
9,SUNTV.NS,14.8213,-8.940476


In [32]:
dict_delta_Smallcap=nifty_delta('SMALLCAP50.csv')
df_nifty_smallcap=pd.DataFrame(dict_delta_Smallcap)
df_nifty_smallcap

C:\Users\PRADEEP\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Stocks,Trailing PE,Delta
0,CANFINHOME.NS,17.2739,11.070855
1,GMMPFAUDLR.NS,76.8822,-3.518007
2,DIXON.NS,171.424,0.161886
3,LAURUSLABS.NS,24.0175,-2.422254
4,WOCKPHARMA.NS,6.37798,-6.835718
5,PERSISTENT.NS,34.0972,2.989703
6,BAJAJELEC.NS,92.3027,-7.955831
7,CHAMBLFERT.NS,7.09609,-0.073960
8,MIDHANI.NS,27.3864,-4.936883
9,BDL.NS,21.6922,-3.760484


In [55]:
df_nifty50=df_nifty50.sort_values(by='Delta',ascending=True)
df_nifty50.reset_index(drop=True,inplace=True)
df_nifty_midcap =df_nifty_midcap.sort_values(by='Delta',ascending=True)
df_nifty_midcap.reset_index(drop=True,inplace=True)
df_nifty_smallcap =df_nifty_smallcap.sort_values(by='Delta',ascending=True)
df_nifty_smallcap.reset_index(drop=True,inplace=True)
df_final=pd.concat([df_nifty_smallcap[:15],df_nifty_midcap[:15],df_nifty50[:10]])
# df_final.reset_index(drop=True,inplace=True)
# df_final=df_final.sort_values(by='Delta',ascending=True)
df_final.reset_index(drop=True,inplace=True)
df_final

,Stocks,Trailing PE,Delta
0,INDIANB.NS,10.9542,-16.151062
1,NBCC.NS,39.6609,-10.384959
2,UJJIVANSFB.NS,NaN,-9.429246
3,VIPIND.NS,NaN,-9.403250
4,IRCON.NS,11.9171,-8.874959
5,SPICEJET.NS,NaN,-8.787523
6,IDFC.NS,NaN,-8.486960
7,SPARC.NS,NaN,-8.176835
8,BAJAJELEC.NS,92.3027,-7.955831
9,RADICO.NS,31.2225,-7.759885


In [17]:
########### 1 week data
name= 'IBREALEST.NS'
ticker = yfin.Ticker(name)
data=ticker.history(start='2021-03-08',end='2021-03-16')
# else:
#     data=ticker.history(period=duration)\
# return data
    # ticker.info
    # ticker.financials
month=data.index.month
month=month.unique() ## to get the unique values
# month=month.value_counts().index

year=data.index.year
year= year.unique() ## to get the unique values
# year=year.sort_values(ascending=True)

three_day_data=[]
for k in range(0,len(month)):
    for l in range(0,len(data)):
        three_day_data.append({'Date':data.index.date[l],'Month':month[k],'Trend':percDiff(data['Open'][l],data['Close'][l])})
            
df=pd.DataFrame(three_day_data).sort_values(by='Date',ascending=True)
df.reset_index(drop=True,inplace=True)
print(df)
print('%age change for the week:',sum(df['Trend']))

         Date  Month     Trend
0  2021-03-08      3 -5.150391
1  2021-03-09      3 -1.639346
2  2021-03-10      3 -0.043481
3  2021-03-12      3 -9.434781
4  2021-03-15      3 -4.476188
%age change for the week: -20.744186817941728


In [ ]:
iCnt=0 ## to count the continuous UP or DOwn trends, 1->3 green candle sticks or 3 red sticks
for k in range(0,5):
    if(k+2<len(df)):
        if(df['Trend'][k+1]<0 and df['Trend'][k+2]<0):
            iCnt+=1
print(iCnt) 

In [ ]:
### TO check the candle stick pattern for a 3/6 month period ##############
name= 'IBREALEST.NS'
start=''
end=''
duration='3mo'
ticker = yfin.Ticker(name)
if (len(start)!= 0 and len(end)!=0):
    data=ticker.history(start=start,end=end)
else:
    data=ticker.history(period=duration)
# return data
    # ticker.info
    # ticker.financials
month=data.index.month
month=month.value_counts().index
year=data.index.year
year= year.value_counts().index
year=year.sort_values(ascending=True)
data

In [ ]:
three_month_data=[]
for k in range(0,len(month)):
    for l in range(0,len(data)):
        if data.index.month[l]== month[k]:
            three_month_data.append({'Date':data.index.date[l],'Month':month[k],'Trend':percDiff(data['Open'][l],data['Close'][l])})

df1=pd.DataFrame(three_month_data).sort_values(by='Date',ascending=True)
df1.reset_index(drop=True,inplace=True)
df1

In [ ]:
iCnt=0 ## to count the continuous UP or DOwn trends, 1->3 green candle sticks or 3 red sticks
for k in range(0,len(df1)):
    if(k+2<len(df1)):
        if(df1['Trend'][k]<0 and df1['Trend'][k+1]<0 and df1['Trend'][k+2]<0):
            print(df1['Trend'][k+2])
            iCnt+=1
         
print(iCnt)     